In [ ]:
# This notebook consists of actions for preparation for `SeismicGeometry` tests
# It creates a fake cube as random noise and convert it into available data formats
import os
import sys
import shutil
import warnings
warnings.filterwarnings('ignore')

import numpy as np

sys.path.insert(0, '../../../seismiqb')

from seismiqb import Geometry, array_to_segy

In [ ]:
# Data creation parameters
CUBE_SHAPE = (1000, 200, 400)
SEED = 42

# Data formats for which running tests
FORMATS = ['sgy', 'qsgy', 'hdf5', 'qhdf5']

# Prepare a workspace

Create directories for files and create a fake cube and save it.

**Storage structure:**
___


**geometry_test_files** (geometry tests directory with temporary files)

&emsp;├── **test_array.npy**

&emsp;└── **test_cube.{DATAFORMAT}**

---
 * **{DATAFORMAT}** means each data format from **FORMATS** list

In [ ]:
OUTPUT_DIR = './geometry_test_files'

# (Re)create the test directory
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(OUTPUT_DIR)

# Create a fake cube

In [ ]:
CUBE_PATH = os.path.join(OUTPUT_DIR, 'test_cube.sgy')

rng = np.random.default_rng(SEED)
data_array = rng.normal(0, 1000, CUBE_SHAPE).astype(np.float32)

with open(os.path.join(OUTPUT_DIR, 'test_array.npy'), 'wb') as outfile:
    np.save(outfile, data_array)

In [ ]:
%%time
array_to_segy(array_like=data_array, path=CUBE_PATH, zip_segy=False, pbar='t')

In [ ]:
%%time
geometry_sgy = Geometry.new(path=CUBE_PATH, collect_stats=True, pbar='t')

loading_error = "Saved and loaded sgy geometry files are different"
assert np.allclose(data_array, geometry_sgy[:, :, :]), loading_error

# Check data loading

In [ ]:
%%time
geometry_sgy = Geometry.new(path=CUBE_PATH)

assert np.allclose(data_array, geometry_sgy[:, :, :]), loading_error

# Conversion

In [ ]:
for data_format in FORMATS:
    if data_format != 'sgy':
        _ = geometry_sgy.convert(format=data_format, quantize=False, store_meta=False, pbar='t')

# Create new sgy from array with spec from the existed sgy

In [ ]:
NEW_CUBE_PATH = os.path.join(OUTPUT_DIR, 'test_new_cube.sgy')

array = np.random.normal(size=geometry_sgy.shape)
array_to_segy(array, NEW_CUBE_PATH, spec=CUBE_PATH, zip_segy=False)
new_geometry_sgy = Geometry.new(NEW_CUBE_PATH, collect_stats=True)

In [ ]:
assert geometry_sgy.equal(new_geometry_sgy), 'Unmatching geometries after export'

Creation with origin

In [ ]:
NEW_CUBE_PATH = os.path.join(OUTPUT_DIR, 'test_new_cube2.sgy')

array = np.random.normal(size=(500, 100, 200))
array_to_segy(array, NEW_CUBE_PATH, spec=CUBE_PATH, origin=(5, 10, 15), zip_segy=False)
new_geometry_sgy = Geometry.new(NEW_CUBE_PATH, collect_stats=False)

assert geometry_sgy.shifts[0] + 5 == new_geometry_sgy.shifts[0], 'Wrong ilines offset'
assert geometry_sgy.shifts[1] + 10 == new_geometry_sgy.shifts[1], 'Wrong xlines offset'

In [ ]:
index_headers = ['INLINE_3D', 'CROSSLINE_3D']

index_ = new_geometry_sgy.headers.set_index(index_headers).index
df = geometry_sgy.headers.set_index(index_headers).loc[index_][['CDP_X', 'CDP_Y']]
new_df = new_geometry_sgy.headers.set_index(index_headers)[['CDP_X', 'CDP_Y']]

assert (df == new_df).all().all(), 'Unmatched `headers`!'